In [1]:
import pandas as pd
import dinuc_shuffle_main 
import pyfaidx
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]
bed_file = "/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/K562/nautilus_runs/K562_02.17.2022_bias_128_4_1234_0.5_fold_0/chrombpnet_model/filtered.peaks.bed"
#model_file = "/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/DNASE_PE/K562/nautilus_runs_may18/K562_05.13.2022_bias_128_4_1234_0.5_fold_0/chrombpnet_model/chrombpnet_wo_bias.h5"
model_file = "/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/K562/nautilus_runs/K562_02.17.2022_bias_128_4_1234_0.5_fold_0/chrombpnet_model/chrombpnet_wo_bias.h5"
output_file = "output/atac_true.h5"
shuffle_flag = False
#output_file = "output/dnase_true.h5"



In [3]:
def softmax(x, temp=1):
    norm_x = x - np.mean(x,axis=1, keepdims=True)
    return np.exp(temp*norm_x)/np.sum(np.exp(temp*norm_x), axis=1, keepdims=True)



In [4]:
bedf = pd.read_csv(bed_file, sep='\t', names=NARROWPEAK_SCHEMA)
gm12878_bed = bedf.sort_values('5',ascending = False).head(100000)

genome_fasta="/mnt/data/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"
genome = pyfaidx.Fasta(genome_fasta)

In [5]:
def get_seq(peaks_df, genome, width, shuffle):
    """
    fetches sequence from a given genome.
    """
    vals = []

    for i, r in peaks_df.iterrows():
        sequence = str(genome[r['chr']][(r['start']+r['summit'] - width//2):(r['start'] + r['summit'] + width//2)])
        if len(sequence) == width:
            if shuffle:
                vals.append(dinuc_shuffle_main.dinuc_shuffle(sequence, num_shufs=1)[0])
            else:
                vals.append(sequence)

    return dinuc_shuffle_main.dna_to_one_hot(vals)

In [6]:
model_chrombpnet_nb = load_model(model_file)

In [7]:
one_hot_seqs_null = get_seq(gm12878_bed, genome, width=2114, shuffle=shuffle_flag)

In [8]:
pred_logits_wo_bias_null, pred_logcts_wo_bias_null = model_chrombpnet_nb.predict([one_hot_seqs_null],
                                      batch_size = 256,
                                      verbose=True)

391/391 [==============================] - 291s 725ms/step


In [9]:
preds_probs_null = softmax(pred_logits_wo_bias_null)

In [10]:
import deepdish as dd
d = {"probs": preds_probs_null}
dd.io.save(output_file, d)